In [96]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

# Metodo Random Forest

In [97]:
df = pd.read_csv("./train.csv")


In [98]:
df['agregados']= df['escuelascercanas']+df['gimnasio']+df['centroscomercialescercanos']+df['piscina']+df['usosmultiples']
train = df.drop(['id','idzona','lat','direccion','ciudad','descripcion','titulo','lng', 'escuelascercanas','gimnasio','centroscomercialescercanos','piscina','usosmultiples'],1)
train['fecha'] = pd.to_datetime(train['fecha']).dt.year
#no acepta nan
train.fillna(0, inplace=True)

In [99]:
X = train
y = train.iloc[:, 8].values 

In [100]:
from sklearn.model_selection import train_test_split
#Separo los datos de "train" en entrenamiento y prueba para probar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [101]:
def mean_target_encoding(train, nombrefeat, nombretarget):
        
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean) #Saco promedio
    temp = preprocessing.scale(temp) #Normalizo
    train[nombrefeat + "_mean"] = temp #Dejo en el DF
    
    return train


In [102]:
def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

In [103]:
X_train = mean_target_encoding(X_train, "tipodepropiedad", "precio")
X_test = mean_target_decoding(X_test, "tipodepropiedad", X_train)
X_train = mean_target_encoding(X_train, "provincia", "precio")
X_test = mean_target_decoding(X_test, "provincia", X_train)

/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [104]:
X_train = X_train.drop(['precio','tipodepropiedad','provincia'],1) 
X_train.fillna(0, inplace=True)
X_test = X_test.drop(['precio','tipodepropiedad','provincia'],1) 
X_test.fillna(0, inplace=True)

In [105]:
X_train

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,fecha,agregados,tipodepropiedad_mean,provincia_mean
138273,20.0,10.0,0.0,0.0,0.0,210.0,2016,2.0,7.446482,0.325665
172283,6.0,3.0,2.0,2.0,165.0,0.0,2015,3.0,0.727875,-0.174906
84608,20.0,4.0,2.0,2.0,210.0,238.0,2015,1.0,-0.409164,0.325665
113718,5.0,3.0,2.0,2.0,90.0,119.0,2012,2.0,1.067938,-0.544324
78170,5.0,2.0,2.0,1.0,113.0,165.0,2016,2.0,-0.409164,0.325665
...,...,...,...,...,...,...,...,...,...,...
119879,5.0,3.0,2.0,3.0,182.0,225.0,2013,3.0,-0.409164,-0.174906
103694,0.0,2.0,1.0,1.0,59.0,59.0,2015,1.0,-0.409164,0.325665
131932,1.0,3.0,2.0,2.0,105.0,105.0,2013,3.0,-0.409164,-0.228784
146867,10.0,4.0,1.0,2.0,80.0,80.0,2016,2.0,-0.409164,0.325665


In [6]:
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True,False],
    'max_depth': [42, 30],
    'min_samples_leaf': [4],
    'min_samples_split': [8, 10],
    'n_estimators': [74, 84, 100],
    'random_state':[42]
}# Create a based model
rf = RandomForestRegressor()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, verbose = 2)
print(grid_search)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'bootstrap': [True, False], 'ma

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV] bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=74 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=74, total=   7.5s
[CV] bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=74 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.5s remaining:    0.0s


[CV]  bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=74, total=   7.4s
[CV] bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=74 
[CV]  bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=74, total=   7.5s
[CV] bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=84 
[CV]  bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=84, total=   8.5s
[CV] bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=84 
[CV]  bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=84, total=   8.4s
[CV] bootstrap=True, max_depth=42, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=84 
[CV]  bootstrap=True, max_depth=42, max_features=2, min_samples_leaf

...

[CV]  bootstrap=True, max_depth=50, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100, total=  13.6s
[CV] bootstrap=True, max_depth=50, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=50, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=100, total=  13.0s
[CV] bootstrap=True, max_depth=50, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=150 
[CV]  bootstrap=True, max_depth=50, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=150, total=  19.8s
[CV] bootstrap=True, max_depth=50, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=150 
[CV]  bootstrap=True, max_depth=50, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=150, total=  19.6s
[CV] bootstrap=True, max_depth=50, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=150 
[CV]  bootstrap=True, max_depth=50, max_features=3, 

[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100, total=  11.8s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100, total=  10.7s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100, total=  11.5s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=150 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=150, total=  16.7s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=150 
[CV]  bootstrap=True, max_depth=80, max_features=2, 

[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=10, n_estimators=84, total=   8.9s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=10, n_estimators=100, total=  10.5s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=10, n_estimators=100, total=  10.8s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=10, n_estimators=100, total=  11.0s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=10, n_estimators=150 
[CV]  bootstrap=True, max_depth=80, max_features=2, m

[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=84, total=   9.2s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=84 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=84, total=   9.2s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=100, total=   9.8s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=100, total=   9.8s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min

[CV]  bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=84, total=  11.7s
[CV] bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=84 
[CV]  bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=84, total=  11.5s
[CV] bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=84 
[CV]  bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=84, total=  11.3s
[CV] bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=100, total=  13.6s
[CV] bootstrap=True, max_depth=80, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=3, min_s

In [ ]:
rf_random.best_params_

In [106]:
# Crear el modelo 
forest = RandomForestRegressor(n_estimators= 74, min_samples_split=10,min_samples_leaf=4,max_features= "sqrt",max_depth=42,bootstrap = False, verbose=2)
#Lo entreno 
forest.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 74
building tree 2 of 74
building tree 3 of 74
building tree 4 of 74
building tree 5 of 74
building tree 6 of 74
building tree 7 of 74
building tree 8 of 74
building tree 9 of 74
building tree 10 of 74
building tree 11 of 74
building tree 12 of 74
building tree 13 of 74
building tree 14 of 74
building tree 15 of 74
building tree 16 of 74
building tree 17 of 74
building tree 18 of 74
building tree 19 of 74
building tree 20 of 74
building tree 21 of 74
building tree 22 of 74
building tree 23 of 74
building tree 24 of 74
building tree 25 of 74
building tree 26 of 74
building tree 27 of 74
building tree 28 of 74
building tree 29 of 74
building tree 30 of 74
building tree 31 of 74
building tree 32 of 74
building tree 33 of 74
building tree 34 of 74
building tree 35 of 74
building tree 36 of 74
building tree 37 of 74
building tree 38 of 74
building tree 39 of 74
building tree 40 of 74
building tree 41 of 74
building tree 42 of 74
building tree 43 of 74
building tree 44 of 

[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:    6.4s finished


RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=42,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=74,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=2, warm_start=False)

In [108]:
Y_pred_forest = forest.predict(X_test)
print('MAE: ', metrics.mean_absolute_error(y_test, Y_pred_forest))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


MAE:  0.013700204864175601


[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:    0.4s finished


## Regresion lineal

In [112]:
 #Defino el algoritmo a utilizar
regresion_lineal = linear_model.LinearRegression()
#Entreno el modelo
regresion_lineal.fit(X_train, y_train)

    
#Realizo una predicción
Y_pred_resgresion = regresion_lineal.predict(X_test)

print('DATOS DEL MODELO REGRESIÓN LINEAL MULTIPLE')
print('Valor de las pendientes o coeficientes "a":')
print(regresion_lineal.coef_)
print('Valor de la intersección o coeficiente "b":')
print(regresion_lineal.intercept_)


DATOS DEL MODELO REGRESIÓN LINEAL MULTIPLE
Valor de las pendientes o coeficientes "a":
[-1.37571063e-15  5.10733134e-16  3.54919452e-16  3.10326636e-16
 -1.04845528e-18  2.40483906e-17  1.00000000e+00  5.47031000e-16
 -4.18259407e-17 -2.25035537e-17]
Valor de la intersección o coeficiente "b":
2.0463630789890885e-12


In [113]:
print('Precisión del modelo:')
print(regresion_lineal.score(X_train, y_train))

Precisión del modelo:
1.0


In [114]:
print('MAE: ', metrics.mean_absolute_error(y_test, Y_pred_resgresion))

MAE:  0.0


## Regresion + forest

In [116]:
solucion = ( Y_pred_resgresion+Y_pred_forest)/2
solucion

array([2013.01067878, 2016.        , 2014.99897625, ..., 2015.99150287,
       2015.00965251, 2012.99620871])

In [117]:
print('MAE: ', metrics.mean_absolute_error(y_test, solucion))

MAE:  0.006850102432075528


In [ ]:
#El mae da muy bajo, revisar que no se este filtrando info con el mean de los feature